

I use sqlalchemy and pandas together because they complement each other very well. Sqlalchemy is a powerful library for working with databases, and it provides a lot of functionality for connecting to and querying databases. Pandas, on the other hand, is a library for working with data in a tabular format, similar to a spreadsheet. By combining these two libraries, I can easily query a database and load the results into a pandas DataFrame for further manipulation and analysis. This makes it easy for me to work with large amounts of data and perform complex data analysis tasks

In [ ]:
!pip3 install mysqlclient
!pip3 install sqlalchemy
!pip3 install plotly 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 KB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysqlclient: filename=mysqlclient-2.1.1-cp38-cp38-linux_x86_64.whl size=102382 sha256=c3f355ac50e7d281e3ddb1cdf4f22182f1c935b615277eb85f9097e9d6c6ccba
  Stored in directory: /root/.cache/pip/wheels/5b/e1/84/a6185eaec318899f59a32d393af7729a0719cd93695d71f9a1
Successfully built mysqlclient
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# SQL test
Importing of librarys and creation of relevant functions

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

def create_connection_url (host, user, password, db):
    """
    Create a connection url for a MySQL database
    
    Parameters:
    host (str): hostname or IP address of the database
    user (str): username for the database
    password (str): password for the database
    db (str): database name
    
    Returns:
    str: a MySQL connection url
    """
    return 'mysql://{}:{}@{}/{}'.format(user, password, host, db)

def make_sql_query(query, conn):
    """
    Execute a SQL query and return the result as a Pandas dataframe
    
    Parameters:
    query (str): SQL query to execute
    conn: a SQLAlchemy engine connection object
    
    Returns:
    pandas.DataFrame: a dataframe containing the result of the SQL query
    """
    return pd.read_sql_query(query, conn)

credentials = {"host": "35.199.127.241",
            "user": "looqbox-challenge",
            "password": "looq-challenge",
            "db" : 'looqbox_challenge'
            }

url = create_connection_url(**credentials)
db_conn = create_engine(url)


## Making Queries

In [ ]:
make_sql_query('show tables', db_conn)

,Tables_in_looqbox_challenge
0,IMDB_movies
1,data_product
2,data_product_sales
3,data_store_cad
4,data_store_sales


### What are the 10 most expensive products in the company?

In [ ]:
q1 = "SELECT PRODUCT_NAME, PRODUCT_VAL FROM looqbox_challenge.data_product ORDER BY PRODUCT_VAL DESC LIMIT 10"
make_sql_query(q1, db_conn)

,PRODUCT_NAME,PRODUCT_VAL
0,Whisky Escoces THE MACALLAN Ruby Garrafa 700ml...,741.99
1,Whisky Escoces JOHNNIE WALKER Blue Label Garra...,735.90
2,Cafeteira Expresso 3 CORACOES Tres Modo Vermelho,499.00
3,Vinho Portugues Tinto Vintage QUINTA DO CRASTO...,445.90
4,Escova Dental Eletrica ORAL B D34 Professional...,399.90
5,Champagne Rose VEUVE CLICQUOT PONSARDIM Garraf...,366.90
6,Champagne Frances Brut Imperial MOET Rose Garr...,359.90
7,Conjunto de Panelas Allegra em Inox TRAMONTINA...,359.00
8,Whisky Escoces CHIVAS REGAL 18 Anos Garrafa 750ml,329.90
9,Champagne Frances Brut Imperial MOET & CHANDON...,315.90


### What sections do the 'BEBIDAS' and 'PADARIA' departments have?


In [ ]:
q2 = "SELECT DISTINCT DEP_NAME, SECTION_NAME, SECTION_COD FROM looqbox_challenge.data_product WHERE DEP_NAME = 'BEBIDAS' OR DEP_NAME = 'PADARIA'"
make_sql_query(q2, db_conn)

,DEP_NAME,SECTION_NAME,SECTION_COD
0,BEBIDAS,BEBIDAS,4
1,BEBIDAS,VINHOS,30
2,PADARIA,DOCES-E-SOBREMESAS,8
3,PADARIA,QUEIJOS-E-FRIOS,22
4,BEBIDAS,CERVEJAS,29
5,PADARIA,PADARIA,19
6,BEBIDAS,REFRESCOS,31
7,PADARIA,GESTANTE,27


### What was the total sale of products (in $) of each Business Area in the first quarter of 2019?


In [ ]:
q3 = """SELECT BUSINESS_NAME, SUM(SALES_QTY) AS Vendas 
        FROM data_store_sales sales INNER JOIN data_store_cad cad 
        ON sales.STORE_CODE = cad.STORE_CODE 
        WHERE DATE BETWEEN '2018-12-31 00:00:00' AND '2019-03-31 23:59:00' 
        GROUP BY BUSINESS_NAME"""
make_sql_query(q3, db_conn)

,BUSINESS_NAME,Vendas
0,Atacado,5259340.0
1,Farma,2876780.0
2,Posto,2367672.0
3,Proximidade,5247398.0
4,Varejo,5280603.0


# Cases

## Case 1

In [ ]:
from typing import Optional

def retrieve_data(product_code: Optional[int] = None, store_code: Optional[int] = None, date: Optional[list] = None):
    """
    Retrieves data from the data_product_sales table in the database based on the provided parameters.
    :param product_code: (Optional) The product code to filter the data by.
    :param store_code: (Optional) The store code to filter the data by.
    :param date: (Optional) A list of two strings representing the start and end date to filter the data by, in ISO-like format.
    :return: A DataFrame containing the retrieved data.
    """
    # Connect to database
    conn = db_conn
    
    # Create the query
    query = "SELECT * FROM data_product_sales WHERE 1=1"
    if product_code:
        query += " AND product_code={}".format(product_code)
    if store_code:
        query += " AND store_code={}".format(store_code)
    if date:
        query += " AND date BETWEEN '{}' AND '{}'".format(date[0], date[1])
    # Execute the query and store the results in a dataframe
    df = make_sql_query(query, conn)
    
    return df



# Example usage 0 
date = ['2019-01-01', '2019-01-31']
mydata0 = retrieve_data(date = date)
print("Example usage 0")
print(mydata0)
# Example usage 1
product_code = 18
store_code = 1

my_data = retrieve_data(product_code, store_code, date)
print("Example usage 1")
print(my_data)

Example usage 0
      STORE_CODE  PRODUCT_CODE        DATE  SALES_VALUE  SALES_QTY
0              1            18  2019-01-01       708.50       65.0
1              1            18  2019-01-02      1297.10      119.0
2              1            18  2019-01-03      1144.50      105.0
3              1            18  2019-01-04      1090.00      100.0
4              1            18  2019-01-05       893.80       82.0
...          ...           ...         ...          ...        ...
39396          9        241404  2019-01-27      5127.75      129.0
39397          9        241404  2019-01-28      4730.25      119.0
39398          9        241404  2019-01-29      4929.00      124.0
39399          9        241404  2019-01-30      9301.50      234.0
39400          9        241404  2019-01-31      7910.25      199.0

[39401 rows x 5 columns]
Example usage 1
   STORE_CODE  PRODUCT_CODE        DATE  SALES_VALUE  SALES_QTY
0           1            18  2019-01-01        708.5       65.0
1         

## Case 2

This is a SQL query that creates a view by joining two tables, data_store_cad and data_store_sales. The first query, q1, selects the columns STORE_CODE, STORE_NAME, START_DATE, END_DATE, BUSINESS_NAME, and BUSINESS_CODE from the data_store_cad table. The second query, q2, selects the columns STORE_CODE, DATE, SALES_VALUE, and SALES_QTY from the data_store_sales table, and filters the results to only show rows where DATE is between '2019-01-01' and '2019-12-31'.

The view query then takes the results of these two queries and joins them on the STORE_CODE column. The query then groups the results by STORE_NAME and BUSINESS_NAME and calculates the total revenue divided by the total quantity sold, rounded to 2 decimal points, and aliases the resulting column as TM. It also filters the results to only show rows where DATE is between '2019-10-01' and '2019-12-31'.

In [ ]:
q1 = """SELECT
      STORE_CODE,
      STORE_NAME,
      START_DATE,
      END_DATE,
      BUSINESS_NAME,
      BUSINESS_CODE
FROM data_store_cad"""

q2 = """SELECT
        STORE_CODE,
        DATE,
        SALES_VALUE,
        SALES_QTY
FROM data_store_sales
WHERE DATE BETWEEN '2019-01-01' AND '2019-12-31'"""

view = """SELECT a.STORE_NAME Loja,
    a.BUSINESS_NAME Categoria,
    ROUND(SUM(b.SALES_VALUE) / SUM(b.SALES_QTY),2) TM
FROM ({}) a
INNER JOIN ({}) b
ON a.STORE_CODE = b.STORE_CODE
WHERE b.DATE BETWEEN '2019-10-01' AND '2019-12-31' GROUP BY Loja, Categoria""".format(q1, q2)
make_sql_query(view, db_conn)

,Loja,Categoria,TM
0,Bahia,Atacado,15.39
1,Bangkok,Posto,13.67
2,Belem,Proximidade,15.37
3,Berlin,Proximidade,15.39
4,Buenos Aires,Atacado,15.39
5,Chicago,Varejo,15.53
6,Dubai,Atacado,15.39
7,Hong Kong,Farma,26.35
8,London,Farma,28.99
9,Madri,Farma,29.03


## Case 3

### We start our analysis by acquiring an overview of the dataset columns and what type of data they hold.

In [ ]:
make_sql_query("SELECT * FROM IMDB_movies", db_conn)

,Id,Title,Genre,Director,Actors,Year,Runtime,Rating,Votes,RevenueMillions,Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.0,757074,333.0,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.0,65.0
2,3,Split,"Horror,Thriller",M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.0,157606,138.0,62.0
3,4,Sing,"Animation,Comedy,Family",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.0,60545,270.0,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.0,393727,325.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.0,27585,NaN,45.0
996,997,Hostel: Part II,Horror,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,6.0,73152,18.0,46.0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.0,70699,58.0,50.0
998,999,Search Party,"Adventure,Comedy",Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,6.0,4881,NaN,22.0


### The first visualization seeks to show the number of films per year and the average revenue calculated. Furthermore, as complementary information to verify the quality of the films, we display the average metascore for each year.

In [ ]:
by_year_query = make_sql_query("SELECT COUNT(Title) Num_titles, AVG(Metascore) Metascore, AVG(RevenueMillions) Revenue, Year FROM IMDB_movies GROUP BY Year", db_conn)

import plotly.express as px
import plotly.graph_objects as go

fig = px.histogram(by_year_query, x='Year', y='Num_titles', labels={'Num_titles':'Number of titles', 'Year':'Year'}, title='Number of titles by year', nbins=by_year_query['Year'].nunique())
fig.add_trace(go.Scatter(x=by_year_query['Year'], y=by_year_query['Metascore'], name='Metascore'))
fig.add_trace(go.Scatter(x=by_year_query['Year'], y=by_year_query['Revenue'], name='Revenue'))
fig.show()


### The next view displays the movies on a coordinate axis formed by the number of votes x revenue in millions. This aims to check if there is any visible correlation between the two parameters. In addition, we bring information about the rating of films by color, to investigate whether there is a correlation between rating and revenue x number of votes. Finally, we scaled the size of each film based on the calculated revenue value, so that we have a more intuitive visualization.

### Note that there is a clear tendency for movies with more votes to have a higher rating. In addition, we can notice indications that the greater the revenue of a film, the greater its number of votes.

In [ ]:

revenue_query = make_sql_query("SELECT Title, Year, Rating, Votes, Metascore, Director, RevenueMillions FROM IMDB_movies", db_conn)
import plotly.express as px
revenue_query.dropna(inplace=True)
# data is the dataframe containing the table above
fig = px.scatter(revenue_query, x='RevenueMillions', y='Votes', size='RevenueMillions', color = 'Rating', hover_name="Title", labels={'RevenueMillions':'Revenue in Millions', 'Votes':'Votes'}, title='Revenue vs Votes')
fig.show()

### Here we bring information related to the quality of a director and his projection, based on the average metascore and revenue associated with him, respectively.

### It is interesting to note that most directors with high metascores are not in the highest levels of revenue

In [ ]:
director_query = make_sql_query("SELECT Director, AVG(Metascore) Metascore, SUM(RevenueMillions) RevenueMillions FROM IMDB_movies GROUP BY Director", db_conn)
director_query.dropna(inplace=True)
fig = px.scatter(director_query, x='Director', y='RevenueMillions', color='Metascore' , size='RevenueMillions', labels={'RevenueMillions':'Revenue in Millions', 'Director':'Director'}, title='Revenue and Metascore by Director')
fig.show()

### Next, we visualize the number of votes received grouped by movie rating. The films that receive the highest amount of votes are those with a rating between 6 and 8. This indicates that films capable of generating the most engagement generally have a rating range within this range. We added a line graph to the histogram to make it easier to see the gradient between each bin in the histogram.

In [ ]:
query_num_votes_per_rating = make_sql_query("SELECT Rating, SUM(Votes) Num_votes FROM IMDB_movies GROUP BY Rating", db_conn)
fig = px.histogram(query_num_votes_per_rating, x='Rating', y='Num_votes', labels={'Num_votes':'Number of votes', 'Rating':'Rating'}, title='Number of votes per rating', nbins=10)
fig.add_trace(go.Scatter(x=query_num_votes_per_rating['Rating'], y=query_num_votes_per_rating['Num_votes'], name = 'Bar'))
fig.show()

### Finally, we bring an analysis of how interrelated are the average rating metrics and metascore for each year. To do so, we divide the metascore by 10 so that it is on the same scale as the rating. This analysis shows us that as of 2008 the two metrics are very different from each other. So much so that between 2008 and 2009 while the rating grew, the metascore decreased. Also, between 2012 and 2014 the metascore decreased, but the rating remained approximately constant.

In [ ]:
query_rating_metrics = make_sql_query("SELECT Year, AVG(Metascore)/10 Metascore, AVG(Rating) Rating FROM IMDB_movies GROUP BY Year", db_conn)
fig = px.line(query_rating_metrics, x='Year', y='Metascore', labels={'Metascore':'Metascore and Rating', 'Year':'Year'}, title='Metascore and Rating per year')
fig.add_trace(go.Scatter(x=query_rating_metrics['Year'], y=query_rating_metrics['Rating'], name = 'Rating'))